In [ ]:
from keras.applications import vgg16

In [ ]:
img_rows, img_cols = 300, 350

model = vgg16.VGG16(weights = 'imagenet',
                   include_top = False,
                   input_shape = (img_rows, img_cols, 3))

for layer in model.layers:
    layer.trainable = False
    
model.summary()

In [ ]:
def add_layer(bottom_model, num_of_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(512, activation = 'relu')(top_model)
    top_model = Dense(num_of_classes, activation = 'softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


num_of_classes = 2
FC_Head = add_layer(model, num_of_classes)

model = Model(inputs = model.input, outputs = FC_Head)
print(model.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "gdrive/MyDrive/dataset"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 45,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    fill_mode = 'nearest')

batch_size = 1

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical')

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint('foot_detector.h15',
                            monitor = 'loss',
                            mode = min,
                            save_best_only = True,
                            verbose = 1)

earlystop = EarlyStopping(monitor = 'loss',
                         min_delta = 0,
                         patience = 3,
                         verbose = 1,
                         restore_best_weights = True)

callbacks = [checkpoint, earlystop]

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'Adam',
             metrics = ['accuracy'])

epochs = 6
#batch_size = 1

history = model.fit_generator(
            train_generator,
            epochs = epochs,
            callbacks = callbacks)

model.save_weights('vggnet_detector.h5')

with open("vggnet_model.json", "w") as json_file:
    json_file.write(model.to_json())